# Initializing the code and models

In [1]:
import os
import pandas as pd
import numpy as np
import vertexai
import json
import requests
import time

from numpy.linalg import norm

from nltk.translate.bleu_score import sentence_bleu

from openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

from google.cloud import aiplatform
from google.oauth2 import service_account
from vertexai.generative_models import GenerativeModel
from vertexai.preview.language_models import TextEmbeddingModel

from langchain_google_vertexai import VertexAI, ChatVertexAI

from langchain.tools import tool
from langchain.pydantic_v1 import BaseModel, Field
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain.agents.tool_calling_agent.base import create_tool_calling_agent
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage

In [2]:
os.environ["AZURE_OPENAI_API_KEY"] = "api-key"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https:..."

In [3]:
credentials = service_account.Credentials.from_service_account_file('service_account_path')

aiplatform.init(project = 'project-id',
                credentials = credentials)

vertexai.init(project="project-id", location="us-central1")

# Getting the data to compare

In [4]:
# Load the data for testing
doc_1_path = "Data/2024_03_04_Summary_test_case_01_27_Mar_2024.xlsx"
doc_2_path = "Data/2024_03_04_Summary_test_case_02_27_March_2024.xlsx"

doc_1 = pd.read_excel(doc_1_path)
doc_2 = pd.read_excel(doc_2_path)

d1_v1 = doc_1.iloc[:, 0]
d1_v2 = doc_1.iloc[:, 1]
d2_v1 = doc_2.iloc[:, 0]
d2_v2 = doc_2.iloc[:, 1]

In [139]:
v1 = """Pharmacovigilance is defined by the World Health Organization as the science and activity related to detecting, assessing, understanding and preventing adverse effects and other medicine-related problems.
The Therapeutic Goods Administration (TGA) collects and evaluates information related to the benefit-risk balance of medicines in Australia to monitor their safety and, where necessary, take appropriate action.
This guidance sets out the pharmacovigilance responsibilities of sponsors of medicines included on the Australian Register of Therapeutic Goods (ARTG) and regulated by the TGA.
It outlines the mandatory reporting requirements and offers recommendations on pharmacovigilance best practice. In this guidance, we use `must` or `required` to describe something you are legally obliged to do.
We use `should` to recommend an action that will assist you to meet your legal requirements. We refer to the TGA as `we` or `us`, and to sponsors as `you`."""

v2 = """Pharmacovigilance is defined by the World Health Organization as the science and activities related to detecting, assessing, understanding and preventing adverse effects and other medicine-related problems.
The Therapeutic Goods Administration (TGA) collects and evaluates information related to the benefit-risk balance of medicines in Australia to monitor their safety and, where necessary, take appropriate action.
This guidance sets out the pharmacovigilance responsibilities of sponsors of medicines included on the Australian Register of Therapeutic Goods (ARTG) and regulated by the TGA.
It outlines the mandatory reporting requirements and offers recommendations on pharmacovigilance best practice. In this guidance we use `must` or `required` to describe something you are legally obliged to do.
We use `should` to recommend an action that will assist you to meet your legal requirements. We reger to the TGA as `we` or `us`, and to sponsors as `you`."""

# Base results (prompt used to get ground-truth version)

In [4]:
prompt_base = f'''Role : As a reviewer of safety and regulatory documents, your responsibility is to compare two versions of a document and identify actionable, noticeable, impactful updates or changes between them.
Instruction:
1. Tag first document {v1}  as V1  and Second document {v2} as V2.
2. Read the content of the sections from V1 and compare with the similar section from V2.
3. Identify actionable, noticeable, impactful updates or changes.
4. Create a precise summary points of identified changes.
Rule : If the identified changes/updates between V1 and V2 are very subtle and don't lead to any noticeable or actionable items, then ignore those changes/updates and don’t include them as part of the summary.
Format: output should be in the form of bullet points only.'''

In [5]:
ans_base = azureOpenAI.chat.completions.create(model="gpt-35-turbo-16k", 
                                               messages=[{"role": "user", "content": prompt_base}])
generated_text = ans_base.choices[0].message.content
print(generated_text)

- Both V1 and V2 have the same content and structure.
- No noticeable or impactful updates or changes were identified between V1 and V2.


# Single Agent

One single agent is in charge of getting the BLEU score, cosine similarity, and getting the list of differences between two versions.

## Creating the tools

In [11]:
@tool
def get_bleu_score(v1:str, v2:str) -> float:
    """Returns the BLEU score between two texts"""
    weights = (0.30, 0.25, 0.25, 0.20)

    bleu = sentence_bleu([v1], v2, weights)
    return round(bleu, 4)

In [12]:
@tool
def get_cosine_similarity_oai(v1:str, v2:str) -> float:
    """Returns the cosine similarity between two texts"""
    embeddings = AzureOpenAIEmbeddings(azure_deployment="text-embedding-3-small",
                                       openai_api_version="2023-05-15")

    emb_1, emb_2 = embeddings.embed_documents([v1, v2])
    
    cosine = np.dot(emb_1, emb_2) / (norm(emb_1) * norm(emb_2))

    return round(cosine, 4)

In [13]:
@tool
def get_cosine_similarity_gcp(v1:str, v2:str) -> float:
    """Returns the cosine similarity between two texts"""
    embeddings = TextEmbeddingModel.from_pretrained("textembedding-gecko")

    emb_1, emb_2 = embeddings.get_embeddings([v1, v2])
    
    cosine = np.dot(emb_1.values, emb_2.values) / (norm(emb_1.values) * norm(emb_2.values))

    return round(cosine, 4)

In [14]:
tools_oai = [get_bleu_score, get_cosine_similarity_oai]
tools_gcp = [get_bleu_score, get_cosine_similarity_gcp]

## Building the prompt

In [5]:
instructions = """"You are a reviewer of safety and regulatory documents. Your responsibility is to compare two versions of a document and identify any impactful and noticeable changes between them.
Instructions:
1. Give me a list of bullet points with all the differences between Version_1 and Version_2.
    1.1. SUMMARIZE the most important changes between Version_1 and Version_2.
    1.2. Be as thorough as possible with explaining the differences in the redaction between Version_1 and Version_2.
    1.3. Compare the redaction between Version_1 and Version_2 ON THE SAME BULLET POINT.
    1.4. Do NOT write the whole text, just the important changes between Version_1 and Version_2.
    1.5. Give me ONLY the bullet points."""

## OpenAI LLM

In [6]:
llm_oai = AzureChatOpenAI(openai_api_version = "2023-05-15",
                          model_name = "gpt-35-turbo-16k",
                          temperature = 0)

## GCP GenAI LLMs

### Gemini

In [7]:
llm_gemini = ChatVertexAI(model_name = 'gemini-1.0-pro-002',
                          max_output_tokens = 4000,
                          temperature = 0)

In [8]:
llm_gemini_15 = ChatVertexAI(model_name = "gemini-1.5-pro-preview-0409",
                             max_output_tokens = 4000,
                             temperature = 0)

### MedLM

In [9]:
llm_medlm = VertexAI(model_name = 'medlm-medium',
                     max_output_tokens = 4000,
                     temperature = 0)

### PaLM-2

In [10]:
llm_palm = VertexAI(model_name = 'text-bison-32k@002',
                    max_output_tokens = 4000,
                    temperature = 0)

## Test with one single pair of texts

Test with one pair of text versions.

In [8]:
v1 = """If the MAH classes a signal as a signal with a serious risk potential ( emerging safety issue) , the following reporting time limits apply: VM-ID: MU101_20_001e_WL - Wegleitung_HD - Hilfsdokument / V6.0 / dst / er / 01.04.2020 3 / 7 The signal must be reported to Swissmedic at once, and at the latest within five days, if measures for maintaining drug safety are required in the short term (e.g. informing the public immediately, market withdrawal at short notice) (Art. 62, para. 2 let. a TPO).
A reporting time limit of 15 days is appropriate if there are other serious drug risks that are not adequately explained in the product information (Art. 62 para. 2 let. b TPO).
It should be noted that emerging safety issues reported for a medicinal product by the MAH to the European Medicines Agency (EMA) are automatically considered, in Switzerland, to be notifiable signals with a serious risk potential, provided the medicinal product/active substance is authorised in Switzerland or an application for authorisation has been submitted to Swissmedic.
The report of an emerging safety issue to the Agency should be accompanied by the available data on the signal in a summary assessment.
In particular, the derived risk-minimising measures and a corresponding timetable for their implementation should also be submitted to the Agency.
If the emerging safety issue is triggered by a single case report in Switzerland, the report of the emerging safety issue including the above-mentioned documentation should be submitted in addition to the report on the adverse drug reaction (ADR).
Following the report of the emerging safety issue , further analyses and investigations of the signal by the marketing authorisation holder and by Swissmedic are usually needed in order to define the definitive measures for risk minimisation.
This takes place in the context of administrative proceedings according to Art. 58 para 3 in conjunction with Art. 66 TPA."""

v2 = """If the MAH classes a signal as a signal with a serious risk potential ( emerging safety issue) , the following reporting time limits apply: The signal must be reported to Swissmedic at once, and at the latest within five days, if measures for maintaining drug safety are required in the short term (e.g. informing the public immediately, market withdrawal at short notice) (Art. 62, para. 2 let. a TPO).
A reporting time limit of 15 days is appropriate if there are other serious drug risks that are not adequately explained in the product information (Art. 62 para. 2 let. b TPO).
It should be noted that emerging safety issues reported for a medicinal product by the MAH to the European Medicines Agency (EMA) are automatically considered, in Switzerland, to be notifiable signals with a serious risk potential, provided the medicinal product/active substance is authorised in Switzerland or an application for authorisation has been submitted to Swissmedic.
Following the report of the emerging safety issue , further analyses and investigations of the signal by the MAH and by Swissmedic are usually needed, in order to define the definitive measures for risk minimisation.
To this end, Swissmedic conducts administrative proceedings according to Art. 58 para. 3 in conjunction with Art. 66 TPA.
The Signal Notification Form should be used for reporting safety signals. The report of an emerging safety issue to the Agency should be accompanied by all the existing available data on the signal in a summary assessment.
In particular, the planned derived risk-minimising measures and a corresponding timetable for their implementation should also be submitted to the Agency.
If this information is incomplete, a date by which further information will be submitted to Swissmedic should be stated.
If the emerging safety issue is triggered by a single case report in Switzerland, the report of the emerging safety issue including the above-mentioned documentation should be submitted in addition to the report on the adverse drug reaction (ADR).
A cross-reference to the report submitted via E2B should be appended to the signal report."""

In [9]:
prompt = ChatPromptTemplate.from_messages([("system", instructions),
                                           ("user", f"Version_1: {v1}\nVersion_2: {v2}"),
                                           MessagesPlaceholder(variable_name = "agent_scratchpad")])

### OpenAI Agent

In [126]:
agent_oai = create_openai_tools_agent(llm_oai, tools_oai, prompt)

executor_oai = AgentExecutor(agent = agent_oai,
                             tools = tools_oai,
                             verbose = True,
                             handle_parsing_errors = True)

In [127]:
executor_oai.invoke({"input": ""})



> Entering new AgentExecutor chain...
- In Version_2, the document starts with the same sentence as Version_1, but the rest of the sentence is different.
- In Version_2, the phrase "VM-ID: MU101_20_001e_WL - Wegleitung_HD - Hilfsdokument / V6.0 / dst / er / 01.04.2020 3 / 7" is removed.
- In Version_2, the sentence "The report of an emerging safety issue to the Agency should be accompanied by the available data on the signal in a summary assessment." is moved to a different position in the document.
- In Version_2, the sentence "In particular, the derived risk-minimising measures and a corresponding timetable for their implementation should also be submitted to the Agency." is moved to a different position in the document.
- In Version_2, the sentence "Following the report of the emerging safety issue , further analyses and investigations of the signal by the marketing authorisation holder and by Swissmedic are usually needed in order to define the definitive measures for risk minimi

{'input': '',
 'output': '- In Version_2, the document starts with the same sentence as Version_1, but the rest of the sentence is different.\n- In Version_2, the phrase "VM-ID: MU101_20_001e_WL - Wegleitung_HD - Hilfsdokument / V6.0 / dst / er / 01.04.2020 3 / 7" is removed.\n- In Version_2, the sentence "The report of an emerging safety issue to the Agency should be accompanied by the available data on the signal in a summary assessment." is moved to a different position in the document.\n- In Version_2, the sentence "In particular, the derived risk-minimising measures and a corresponding timetable for their implementation should also be submitted to the Agency." is moved to a different position in the document.\n- In Version_2, the sentence "Following the report of the emerging safety issue , further analyses and investigations of the signal by the marketing authorisation holder and by Swissmedic are usually needed in order to define the definitive measures for risk minimisation." i

# Test with multiple pairs

Execute all LLM agents through the whole document

In [11]:
def build_oai_agent(v1:str, v2:str):
    # Prepare the LLM
    llm_oai = AzureChatOpenAI(openai_api_version = "2023-05-15",
                              model_name = "gpt-35-turbo-16k",
                              temperature = 0)

    # Build the tools for the LLM agent
    tools = [get_bleu_score, get_cosine_similarity_oai]

    # Establish the prompt for the agent
    prompt = ChatPromptTemplate.from_messages([("system", instructions),
                                               ("user", f"V1: {v1}\nV2: {v2}"),
                                               MessagesPlaceholder(variable_name = "agent_scratchpad")])

    # Build the agent
    agent = create_openai_tools_agent(llm_oai, tools, prompt)
    return agent

In [12]:
def build_gcp_agent(v1:str, v2:str, model:str):
    # Prepare the LLM
    llm_gemini = ChatVertexAI(model_name = model,
                              max_output_tokens = 4000,
                              temperature = 0)

    # Build the tools for the LLM agent
    tools = [get_bleu_score, get_cosine_similarity_gcp]

    # Establish the prompt for the agent
    prompt = ChatPromptTemplate.from_messages([("system", instructions),
                                               ("user", f"V1: {v1}\nV2: {v2}"),
                                               MessagesPlaceholder(variable_name = "agent_scratchpad")])

    # Build the agent
    gemini_tools = llm_gemini.bind_tools(tools)

    agent = create_tool_calling_agent(gemini_tools, tools, prompt)
    return agent

## Running OpenAI agent

In [ ]:
oai_ans = []

# Get the results from each pair of text
for i in range(len(d1_v1)):
    print(f"Pair {i+1} of {len(d1_v1)}...", end = '\r')
    agent_oai = build_oai_agent(d1_v1[i], d1_v2[i])

    # Execute the agent
    executor = AgentExecutor(agent = agent_oai,
                             tools = tools_oai,
                             verbose = True,
                             handle_parsing_errors = True)

    oai_ans.append(executor.invoke({"input": ''})['output'])

## Call the Gemini LLM

In [13]:
oai_ans = []

for i in range(len(d1_v1)):
    print(f"Pair {i+1} of {len(d1_v1)}...", end = '\r')
    
    prompt = instructions + f"""\n\nVersion_1: '{d1_v1[i]}'\nVersion_2: '{d1_v2[i]}'"""

    oai_ans.append(llm_oai.invoke(prompt).content)

In [18]:
gemini_15_ans = []

for i in range(len(d1_v1)):
    print(f"Pair {i+1} of {len(d1_v1)}...", end = '\r')
    
    prompt = instructions + f"""\n\nVersion_1: '{d1_v1[i]}'\nVersion_2: '{d1_v2[i]}'"""

    gemini_15_ans.append(llm_gemini_15.invoke(prompt).content)

    time.sleep(30)

In [19]:
gemini_ans = []

for i in range(len(d1_v1)):
    print(f"Pair {i+1} of {len(d1_v1)}...", end = '\r')
    
    prompt = instructions + f"""\n\nVersion_1: '{d1_v1[i]}'\nVersion_2: '{d1_v2[i]}'"""

    gemini_ans.append(llm_gemini.invoke(prompt).content)

In [20]:
palm_ans = []

for i in range(len(d1_v1)):
    print(f"Pair {i+1} of {len(d1_v1)}...", end = '\r')
    
    prompt = instructions + f"""\n\nVersion_1: '{d1_v1[i]}'\nVersion_2: '{d1_v2[i]}'"""

    palm_ans.append(llm_palm.invoke(prompt))

In [ ]:
medlm_ans = []

for i in range(len(d1_v1)):
    print(f"Pair {i+1} of {len(d1_v1)}...", end = '\r')
    
    prompt = instructions + f"""\n\nVersion_1: '{d1_v1[i]}'\nVersion_2: '{d1_v2[i]}'"""

    medlm_ans.append(llm_medlm.invoke(prompt))

## Evaluate the results

In [22]:
# Build the dataframe with the results
agent_df = pd.DataFrame(data = {"V1": d1_v1,
                                "V2": d1_v2,
                                "Orig_Ans": doc_1.iloc[:, 3],
                                "Base_comments": doc_1.iloc[:, 4],
                                "OAI_ans": oai_ans,
                                "Gemini_15_ans": gemini_15_ans,
                                "Gemini_10_ans": gemini_ans,
                                "MedLM_ans": medlm_ans,
                                "PaLM_ans": palm_ans})
agent_df.head(3)

,V1,V2,Orig_Ans,Base_comments,OAI_ans,Gemini_15_ans,Gemini_10_ans,MedLM_ans,PaLM_ans
0,Title: Guidance document - Drug Safety Signals...,Title: HD-Guidance document\nGuidance document...,NaN,Change not seen by the Gen AI\nHad an impcat o...,- The title of Version_2 has been changed to '...,## Version Comparison: Drug Safety Signals Gui...,## Differences between Version_1 and Version_2...,**Summary of the most important changes betwe...,- **Title:**\n - Version_1: 'Guidance docume...
1,The duty to report drug safety signals and the...,The duty to report drug safety signals and the...,Summary of Changes between V1 and V2: \n1. The...,Comment is meaningless\nSection header woudm b...,"- The document ""GVP Annex I - Definitions (Rev...",## Version 1 vs. Version 2: Key Differences\n\...,## Differences between Version_1 and Version_2...,**Summary of the most important changes betwe...,**Changes between Version_1 and Version_2:**\...
2,"In the context of signal and risk management, ...","In the context of signal and risk management, ...",Summary of Changes between V1 and V2:\n1. The ...,NaN,"- The abbreviation ""MAH"" is used instead of ""M...",## Version 1 vs. Version 2: Redaction Comparis...,## Differences between Version_1 and Version_2...,There are no differences between Version_1 an...,**Differences between Version_1 and Version_2...
